### Add papers from .json files

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import collections
data_path = "../../data/"
data = pd.read_excel(os.path.join(data_path,'database.xlsx'), sheet_name='Details')
data_R2 = pd.read_excel(os.path.join(data_path,'database.xlsx'), sheet_name='R^2')

Adding information to the meta-analysis

In [36]:
'''
if 'Vendor' not in data: data.insert(5, 'Vendor', np.NaN, allow_duplicates = False)
if 'Neutral sentiment confidence: title + abstract' not in data: data['Neutral sentiment confidence: title + abstract'] = np.NaN
if 'Positive sentiment confidence: title + abstract' not in data: data['Positive sentiment confidence: title + abstract'] = np.NaN
if 'Negative sentiment confidence: title + abstract' not in data: data['Negative sentiment confidence: title + abstract'] = np.NaN
if 'Mixed sentiment confidence: title + abstract' not in data: data['Mixed sentiment confidence: title + abstract'] = np.NaN
if 'Neutral sentiment confidence: title + abstract + text' not in data: data['Neutral sentiment confidence: title + abstract + text'] = np.NaN
if 'Positive sentiment confidence: title + abstract + text' not in data: data['Positive sentiment confidence: title + abstract + text'] = np.NaN
if 'Negative sentiment confidence: title + abstract + text' not in data: data['Negative sentiment confidence: title + abstract + text'] = np.NaN
if 'Mixed sentiment confidence: title + abstract + text' not in data: data['Mixed sentiment confidence: title + abstract + text'] = np.NaN
'''

"\nif 'Vendor' not in data: data.insert(5, 'Vendor', np.NaN, allow_duplicates = False)\nif 'Neutral sentiment confidence: title + abstract' not in data: data['Neutral sentiment confidence: title + abstract'] = np.NaN\nif 'Positive sentiment confidence: title + abstract' not in data: data['Positive sentiment confidence: title + abstract'] = np.NaN\nif 'Negative sentiment confidence: title + abstract' not in data: data['Negative sentiment confidence: title + abstract'] = np.NaN\nif 'Mixed sentiment confidence: title + abstract' not in data: data['Mixed sentiment confidence: title + abstract'] = np.NaN\nif 'Neutral sentiment confidence: title + abstract + text' not in data: data['Neutral sentiment confidence: title + abstract + text'] = np.NaN\nif 'Positive sentiment confidence: title + abstract + text' not in data: data['Positive sentiment confidence: title + abstract + text'] = np.NaN\nif 'Negative sentiment confidence: title + abstract + text' not in data: data['Negative sentiment conf

Adding new papers through .json files storing core information

In [34]:
original_sheet_shape = [data.shape,data_R2.shape]
change=0
str_keys_to_int = ['Year','Subjects','ROI per subject']
str_keys_to_float = ['Magnetic field']
for file in os.listdir("../../binder/new-paper-json-files/"):
    if file.endswith(".json"): 
        with open(f'../../binder/new-paper-json-files/{file}', 'r') as f:
            new_paper_json = json.load(f)
            if data['DOI'].str.contains(new_paper_json['DOI']).any(): print(f"The {new_paper_json['First author']} et al. {new_paper_json['Year']} paper has already been uploaded to the spreadsheet!")
            else:
                change+=1
                
                # update R^2 sheet
                new_paper_R2_dict = dict(zip(data_R2.columns,[np.NaN]*len(data_R2.columns)))
                new_paper_R2_dict['DOI'] = new_paper_json['DOI']
                for i in range(len(new_paper_json['R^2'])): new_paper_R2_dict[new_paper_json['R^2'][i]['MRI measure being compared to histology']] = float(new_paper_json['R^2'][i]['Value'])
                data_R2 = pd.concat([data_R2,pd.DataFrame([new_paper_R2_dict])],ignore_index=True)
                
                # update Details sheet
                new_paper_json.pop('R^2')
                for key in str_keys_to_int: new_paper_json[key] = int(new_paper_json[key])
                for key in str_keys_to_float: new_paper_json[key] = float(new_paper_json[key])
                if (collections.Counter(list(data.columns)) == collections.Counter(list(new_paper_json.keys()))):
                    data = pd.concat([data,pd.DataFrame([new_paper_json])],ignore_index=True)
                else: 
                    print("Could not add to data; different col names")

Could not add to data; different col names
Could not add to data; different col names
Could not add to data; different col names
Could not add to data; different col names


Rewriting `database.xlsx` if new paper .json files

In [35]:
if (change != 0):
    Excelwriter = pd.ExcelWriter(os.path.join(data_path,'database.xlsx'),engine="xlsxwriter")
    data.to_excel(Excelwriter, sheet_name='Details',index=False)
    data_R2.to_excel(Excelwriter, sheet_name='R^2',index=False)
    Excelwriter.save()
    print(f"{change} papers have been added.\nYour details spreadsheet went from shape {original_sheet_shape[0]} to {data.shape}.\nYour R^2 spreadsheet went from shape {original_sheet_shape[1]} to {data_R2.shape}.")
else:
    print("No new papers to add!")

4 papers have been added.
Your details spreadsheet went from shape (58, 28) to (58, 28).
Your R^2 spreadsheet went from shape (58, 36) to (62, 36).
